In [14]:
using ModelingToolkit, DifferentialEquations, Plots, LinearAlgebra
using ModelingToolkit: t_nounits as t, D_nounits as D

#An attempt to use vecorized representation for the standard Lotka-Volterra model.

#First build the symbolic model

#Set the number of patches
Np=2
#Set the number of corridors
Nc=1

#patches
@variables p(t)[1:Np]
#corridors
@variables c(t)[1:Np,1:Np] #note, this is only for 1 corridor and will have to be modifed.

#=
Note, think about generalizing c(t) for multiple corridors... i.e. c(t)[1:Nc,1:Np,1:Np]
I have not tested this, but this shold make a Nc x Np x Np array.
=#


@parameters α[1:Nc] 
@parameters β[1:Nc] 
@parameters d[1:Nc]

#=

Note, think about generalizing the above for directions, i.e. add more dimensions
to the array where, again, there will be lots of zeros...

=#

#=
define a general flux at a node connection (between a patch and a corridor),
that is, the flux entering and exiting corridors. All fluxes are defined for 
corridors, not patches.  Net Fluxes for patches are then summed over all corridors
connected to a patch, i.e. sum in - sum out. This should allow us to write the patch 
fluxes and corridor fluxes in a very systematic way, and we should end up with 
something like

D.(p) = NPF
D.(c) = NCF

where NPF and NCF are the net patch and corridor fluxes appropriately defined.

=#

EnFlx(p,c,d,α,β) = d * exp(-α * β * c) * p
ExFlx(c,β) = exp(-β*c)*c

#=
so, for example, the fluxes from p1 and p2 into corridor 1 would be

EnFlx(p[1],c[1,2],d[1],α[1],β[1])
EnFlx(p[2],c[2,1],d₂,α₂,β₂)

etcetera

=#

#define the corridor flux matrix

eqs = [
    D.(p[1]) ~ ExFlx(c[2,1],β[1]) - EnFlx(p[1],c[1,2],d[1],α[1],β[1]),
    D.(p[2]) ~ ExFlx(c[1,2],β[1]) - EnFlx(p[2],c[2,1],d[1],α[1],β[1]),
    D.(c[1,2]) ~ - ExFlx(c[1,2],β[1]) + EnFlx(p[1],c[1,2],d[1],α[1],β[1]),
    D.(c[2,1]) ~ - ExFlx(c[2,1],β[1]) + EnFlx(p[2],c[2,1],d[1],α[1],β[1]) 
]

#define equations

#eqs = [
 #   D.(p) ~ collect(c*p)
  #      ]

#build model symbolically
@mtkbuild model = ODESystem(eqs, t)


Model model:
Equations (4):
  4 standard: see equations(model)
Unknowns (4): see unknowns(model)
  (c(t))[1, 2]
  (c(t))[2, 1]
  (p(t))[1]
  (p(t))[2]
Parameters (3): see parameters(model)
  α
  β
  d

OK, so this works beautifully, and if you stare at it for a while you should be able to generalize. But first, create the numerical problem and solve it... Notice that it didn't complain about us not using c[1,1] and c[2,2]. But when we generalize, these will all be there, but set to zero.  Maybe the best way to proceed is to just have this file with subseqent cells as incremental generalizations.